In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
api_key_openai = os.environ.get('OPENAI_API_KEY')
org_openai = os.environ.get('OPENAI_ORG')

import openai
openai.organization = org_openai
import numpy as np

In [3]:
import os
import sys
import inspect

sys.path.insert(0, '../..')

import pickle

In [4]:
from src.GetEmbeddings.GetEmbeddingsOpenAI import GetEmbeddingsOpenAI

In [5]:
from src.Clustering.Clustering import Clustering_and_DimRed

c:\Users\arik_\anaconda3\envs\llm_sem\Lib\site-packages\umap\distances.py:1063: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
c:\Users\arik_\anaconda3\envs\llm_sem\Lib\site-packages\umap\distances.py:1071: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
c:\Users\arik_\anaconda3\envs\llm_sem\Lib\site-packages\umap\distances.py:1086: NumbaD

In [6]:
data = pickle.load(open("../../Data/Emebeddings/embeddings_20ng_raw.pkl", "rb"))
embeddings = data["embeddings"]
corpus = data["corpus"]

In [7]:
vocab_embeddings = pickle.load(open("../../Data/Emebeddings/embeddings_20ng_vocab.pkl", "rb"))

In [8]:
clusterer = Clustering_and_DimRed(min_cluster_size_hdbscan = 5, number_clusters_hdbscan = 20)
dim_red_embeddings, clusters, umap_mapper = clusterer.cluster_and_reduce(embeddings)

UMAP(angular_rp_forest=True, metric='cosine', min_dist=0, n_components=5, random_state=42, verbose=True)
Fri Aug  4 10:09:27 2023 Construct fuzzy simplicial set
Fri Aug  4 10:09:27 2023 Finding Nearest Neighbors
Fri Aug  4 10:09:27 2023 Building RP forest with 12 trees
Fri Aug  4 10:09:33 2023 NN descent for 14 iterations
	 1  /  14
	 2  /  14
	 3  /  14
	 4  /  14
	 5  /  14
	 6  /  14
	Stopping threshold met -- exiting after 6 iterations
Fri Aug  4 10:09:57 2023 Finished Nearest Neighbor Search
Fri Aug  4 10:10:00 2023 Construct embedding


Epochs completed: 100%| ██████████ 200/200 [00:13]


Fri Aug  4 10:10:14 2023 Finished embedding


In [9]:
clusterer.visualize_clusters_dynamic(dim_red_embeddings, clusters, corpus)

In [10]:
#visuslize clusters
#clusterer.visualize_clusters_dynamic(dim_red_embeddings, clusters, texts= corpus)

In [11]:
import ExtractTopWords
extractor = ExtractTopWords.ExtractTopWords()

c:\Users\arik_\Documents\Dokumente\Studium\Master\LLM_seminar\Project\src


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\arik_\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\arik_\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [12]:
centroid_dict = extractor.extract_centroids(dim_red_embeddings, clusters)

In [13]:
extractor.compute_centroid_similarity(dim_red_embeddings, extractor.extract_centroids(dim_red_embeddings, clusters), 0)

array([0.00610244, 0.0072777 , 0.00557646, ..., 0.00731738, 0.00708943,
       0.00680215], dtype=float32)

In [14]:
vocab = extractor.compute_corpus_vocab(corpus, min_freq = 0.3, max_freq=0.9)

Processing corpus: 100%|██████████| 18466/18466 [00:32<00:00, 567.34it/s]


Most common words:
n't: 17117
would: 10872
one: 10144
people: 6442
like: 6409
get: 5815
know: 5742
also: 5589
use: 4989
think: 4981


In [15]:
len(vocab)

20775

In [16]:
words_per_topic = extractor.compute_words_topics(corpus, vocab, clusters)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\arik_\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
Computing words per topic: 100%|██████████| 18466/18466 [00:19<00:00, 957.62it/s] 


In [17]:
np.unique(clusters)

array([-1,  0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15,
       16, 17, 18, 19], dtype=int64)

In [18]:
tws, tfidfs = extractor.extract_topics_tfidf(corpus, vocab, clusters, top_n_words=10)

Computing word-topic matrix: 100%|██████████| 18466/18466 [00:55<00:00, 331.81it/s]

(20775, 20) (20775,)



c:\Users\arik_\Documents\Dokumente\Studium\Master\LLM_seminar\Project\src\ExtractTopWords\ExtractTopWords.py:257: RuntimeWarning:

invalid value encountered in divide

c:\Users\arik_\Documents\Dokumente\Studium\Master\LLM_seminar\Project\src\ExtractTopWords\ExtractTopWords.py:258: RuntimeWarning:

divide by zero encountered in divide

c:\Users\arik_\Documents\Dokumente\Studium\Master\LLM_seminar\Project\src\ExtractTopWords\ExtractTopWords.py:262: RuntimeWarning:

invalid value encountered in multiply



In [20]:
tws2, cosine_similarity= extractor.extract_top_words_centroid_similarity(vocab, vocab_embeddings, centroid_dict, umap_mapper)

Fri Aug  4 10:12:43 2023 Worst tree score: 0.35134842
Fri Aug  4 10:12:43 2023 Mean tree score: 0.36268457
Fri Aug  4 10:12:43 2023 Best tree score: 0.37967075
Fri Aug  4 10:12:47 2023 Forward diversification reduced edges from 276990 to 99863
Fri Aug  4 10:12:51 2023 Reverse diversification reduced edges from 99863 to 99863


c:\Users\arik_\anaconda3\envs\llm_sem\Lib\site-packages\scipy\sparse\_index.py:143: SparseEfficiencyWarning:

Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.



Fri Aug  4 10:12:55 2023 Degree pruning reduced edges from 114512 to 112287
Fri Aug  4 10:12:55 2023 Resorting data and graph based on tree order
Fri Aug  4 10:12:55 2023 Building and compiling search function


Epochs completed: 100%| ██████████ 30/30 [00:02]


In [29]:
tws2[10]

['cryptographically',
 'cypher',
 'decrypts',
 'non-clipper',
 'decrypting',
 'cryptologic',
 'triple-encrypt',
 'encrypts',
 'cryptographer',
 'clipper-chip']